In [2]:
import yfinance as yf

token = 'BTC-USD'
# start_date = '2015-01-01'

data = yf.download(token, interval='1d')
data.to_csv(f'{token}.csv')
data

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-09-17,465.864014,468.174011,452.421997,457.334015,457.334015,21056800
2014-09-18,456.859985,456.859985,413.104004,424.440002,424.440002,34483200
2014-09-19,424.102997,427.834991,384.532013,394.795990,394.795990,37919700
2014-09-20,394.673004,423.295990,389.882996,408.903992,408.903992,36863600
2014-09-21,408.084991,412.425995,393.181000,398.821014,398.821014,26580100
...,...,...,...,...,...,...
2024-12-21,97756.195312,99507.101562,96426.523438,97224.726562,97224.726562,51765334294
2024-12-22,97218.320312,97360.265625,94202.187500,95104.937500,95104.937500,43147981314
2024-12-23,95099.390625,96416.210938,92403.132812,94686.242188,94686.242188,65239002919


In [8]:
import pandas as pd 

data = pd.read_csv(f'{token}.csv', parse_dates=['Date'], index_col=['Date'])
data.drop(['Adj Close', 'Open', 'High', 'Low', 'Volume'], axis=1, inplace=True)
data

,Close
Date,
2014-09-17,457.334015
2014-09-18,424.440002
2014-09-19,394.795990
2014-09-20,408.903992
2014-09-21,398.821014
...,...
2024-12-21,97224.726562
2024-12-22,95104.937500
2024-12-23,94686.242188


In [3]:
import pandas as pd 
import ta

# data['sma'] = ta.trend.sma_indicator(data['Close'], window=30)
data['ema20'] = ta.trend.ema_indicator(data['Close'], window=20)
data['ema50'] = ta.trend.ema_indicator(data['Close'], window=50)
# data['ema200'] = ta.trend.ema_indicator(data['Close'], window=200)


# Añadir RSI
data['rsi'] = ta.momentum.rsi(data['Close'], window=14)

# Añadir MACD
macd = ta.trend.MACD(data['Close'], window_slow=26, window_fast=12, window_sign=9)
data['macd'] = macd.macd()
data['macd_signal'] = macd.macd_signal()

#Precio Futuro
# data['Close_future'] = data['Close'].shift(-1)
# Dirección del Movimiento de Precio
data['Price_Up'] = (data['Close'].shift(-1) > data['Close']).astype(int)


#Boramos datos nulos
data.dropna(inplace=True)

data.to_csv('BTC-I.csv',index=['Date'])
data = pd.read_csv('BTC-I.csv', parse_dates=['Date'], index_col=['Date'])
data

,Open,High,Low,Close,Adj Close,Volume,ema20,ema50,rsi,macd,macd_signal,Price_Up
Date,,,,,,,,,,,,
2014-11-05,330.683014,343.368988,330.683014,339.485992,339.485992,19817200,349.705580,374.193837,41.651927,-13.723550,-12.462599,1
2014-11-06,339.458008,352.966003,338.424011,349.290009,349.290009,18797000,349.666002,373.217216,46.905741,-12.079876,-12.386054,0
2014-11-07,349.817993,352.731995,341.776001,342.415009,342.415009,16834200,348.975431,372.009287,43.919280,-11.202868,-12.149417,1
2014-11-08,342.153992,347.032013,342.153992,345.488007,345.488007,8535470,348.643295,370.969237,45.586943,-10.142946,-11.748123,1
2014-11-09,345.376007,363.626007,344.255005,363.264008,363.264008,24205600,350.035744,370.667071,54.091375,-7.778906,-10.954279,1
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-21,97756.195312,99507.101562,96426.523438,97224.726562,97224.726562,51765334294,99069.601344,92289.965450,48.952697,1965.864891,3012.484494,0
2024-12-22,97218.320312,97360.265625,94202.187500,95104.937500,95104.937500,43147981314,98692.014312,92400.356511,44.934433,1437.643624,2697.516320,0
2024-12-23,95099.390625,96416.210938,92403.132812,94686.242188,94686.242188,65239002919,98310.512205,92489.999087,44.163328,974.011455,2352.815347,1


In [6]:
features = ['Close', 'sma', 'ema20', 'ema50', 'ema200', 'rsi', 'macd', 'macd_signal']
target = ['Price_Up']
# Separar X e y
X = data[features]
y = data[target]


In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01, shuffle= False)


ValueError: With n_samples=0, test_size=0.01 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
